In [130]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
# import collections
# import torch
# from torch import nn
# from torch.utils.data import Dataset, DataLoader, TensorDataset

In [2]:
#Benchmarking PCA model

# GET Input Feature Data

In [123]:
zacks_fc = pd.read_csv("../data/fundamental_data/ZACKS_FC.txt", delimiter=',')
zacks_fc.head()
# zacks_fc['per_end_date'] = pd.to_datetime(zacks_fc['per_end_date'])
# zacks_fc['per_end_date'] = zacks_fc['per_end_date'].dt.tz_localize('US/Eastern')
# zacks_fc['filing_date'] = pd.to_datetime(zacks_fc['filing_date'])
# zacks_fc['filing_date'] = zacks_fc['filing_date'].dt.tz_localize('US/Eastern')
# zacks_fc = zacks_fc.sort_values('per_end_date').set_index('per_end_date')

/var/folders/vf/gw1t6f4j3mv84kp8kpw4j_bw0000gn/T/ipykernel_60214/513894337.py:1: DtypeWarning: Columns (4,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,44,45,46,47,49,50,51,52,53,54,55,56,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  zacks_fc = pd.read_csv("../data/fundamental_data/ZACKS_FC.txt", delimiter=',')


,m_ticker,ticker,comp_name,comp_name_2,exchange,currency_code,per_end_date,per_type,per_code,per_fisc_year,...,stock_based_compsn_qd,cash_flow_oper_activity_qd,net_change_prop_plant_equip_qd,comm_stock_div_paid_qd,pref_stock_div_paid_qd,tot_comm_pref_stock_div_qd,wavg_shares_out,wavg_shares_out_diluted,eps_basic_net,eps_diluted_net
0,YSTI,V.NTS,NaN,NANOTECH SECRTY,NaN,CND,2015-03-31,Q,NaN,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,48.751,48.751,NaN,NaN
1,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2006-12-31,A,NaN,2006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2007-12-31,A,NaN,2007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2008-12-31,A,NaN,2008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2009-12-31,A,NaN,2009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
tickers_list = zacks_fc["ticker"].unique()

In [60]:
import wrds
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Connect to WRDS
db = wrds.Connection()

# Define start and end dates
start_date = '1990-01-01'
end_date = '2025-01-01'



WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [103]:

query_crsp = f"""
    SELECT msf.*,
           msi.*,  -- Explicitly select columns from msi to avoid ambiguity
           msenames.ticker
    FROM crsp.msf AS msf
    LEFT JOIN crsp.msi AS msi ON msf.date = msi.date
    LEFT JOIN crsp.msenames AS msenames 
        ON msf.permno = msenames.permno 
        AND msenames.namedt <= msf.date 
        AND msf.date <= msenames.nameendt
    WHERE msf.date BETWEEN '{start_date}' AND '{end_date}'
"""
crsp_data = db.raw_sql(query_crsp, date_cols=['date'])

crsp_data.to_csv('../data/wrds_features_all.csv', index=False)
db.close()

In [4]:
crsp_data = pd.read_csv('../data/wrds_features_all.csv')

In [5]:
crsp_data.head()

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,vwretx,ewretd,ewretx,sprtrn,spindx,totval,totcnt,usdval,usdcnt,ticker
0,00621210,10065,20023,0,1,6722.0,2009-02-27,6.7087,8.04,6.710,...,-0.103353,-0.107606,-0.109708,-0.109931,735.09,9.985858e+09,6710,1.116577e+10,6698,ADX
1,00621210,10065,20023,0,1,6722.0,2009-03-31,6.0620,7.78,7.522,...,0.083548,0.107474,0.104391,0.085405,797.87,1.073418e+10,6665,9.893487e+09,6658,ADX
2,00621210,10065,20023,0,1,6722.0,2009-04-30,7.4900,8.15,8.150,...,0.107632,0.192762,0.190929,0.093925,872.81,1.190017e+10,6635,1.071496e+10,6617,ADX
3,00621210,10065,20023,0,1,6722.0,2009-05-29,8.1000,8.57,8.440,...,0.065137,0.102329,0.100210,0.053081,919.14,1.274711e+10,6598,1.189937e+10,6585,ADX
4,00621210,10065,20023,0,1,6722.0,2009-06-30,8.2100,8.70,8.400,...,-0.004962,0.023921,0.021352,0.000196,919.32,1.274607e+10,6575,1.274197e+10,6545,ADX


In [6]:
crsp_data.dtypes

cusip        object
permno        int64
permco        int64
issuno        int64
hexcd         int64
hsiccd      float64
date         object
bidlo       float64
askhi       float64
prc         float64
vol         float64
ret         float64
bid         float64
ask         float64
shrout      float64
cfacpr      float64
cfacshr     float64
altprc      float64
spread      float64
altprcdt     object
retx        float64
date.1       object
vwretd      float64
vwretx      float64
ewretd      float64
ewretx      float64
sprtrn      float64
spindx      float64
totval      float64
totcnt        int64
usdval      float64
usdcnt        int64
ticker       object
dtype: object

In [7]:
crsp_data.dropna(subset=['ticker'], inplace=True)

In [8]:
crsp_data['mktcap'] = crsp_data['prc'].abs()*crsp_data['cfacpr'] * crsp_data['shrout'] * crsp_data['cfacshr']  # Market Cap
crsp_data['turnover'] = crsp_data['vol'] / crsp_data['shrout']  # Turnover
crsp_data['adjprc'] = crsp_data['prc']*crsp_data['cfacpr']


In [9]:
crsp_data['prc'].isna().sum()

np.int64(1816)

In [113]:
query = f"""
    SELECT date, rf 
    FROM ff.factors_monthly
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
"""

# Retrieve data
t_bill_data = db.raw_sql(query, date_cols=['date'])
t_bill_data.head()

,date,rf
0,1990-01-01,0.0057
1,1990-02-01,0.0057
2,1990-03-01,0.0064
3,1990-04-01,0.0069
4,1990-05-01,0.0068


In [127]:
len(crsp_data[crsp_data.ticker=='V.NTS'])

0

In [131]:
concat_list = []
for ticker in tqdm(tickers_list, desc="cleaning", unit="ticker"):
    temp = crsp_data[crsp_data.ticker==ticker]
    if len(temp)==0:
        continue
    temp = temp.drop_duplicates(subset=['date'],keep='last')
    temp['date'] = pd.to_datetime(temp['date'])
    temp = temp.sort_values(by='date', ascending=True) 
    # print(temp.date)
    temp = pd.merge_asof(
        temp,
        t_bill_data,
        on='date',
        direction='backward'
    )
    concat_list.append(temp)
crsp_data = pd.concat(concat_list,ignore_index=True)   


cleaning: 100%|██████████| 16688/16688 [39:03<00:00,  7.12ticker/s]


In [149]:
crsp_data.to_csv("../data/wrds_processed_features.csv")

Sanity Check

In [140]:
unique_dates = sorted(crsp_data['date'].unique())


date_diffs = pd.Series([unique_dates[i+1] - unique_dates[i] for i in range(len(unique_dates) - 1)])

average_diff = date_diffs.mean()

print(f"Average difference between dates: {average_diff}")
pd.DataFrame(date_diffs).describe()

Average difference between dates: 30 days 10:28:55.560859188


,0
count,419
mean,30 days 10:28:55.560859188
std,1 days 11:56:03.733891209
min,28 days 00:00:00
25%,29 days 00:00:00
50%,31 days 00:00:00
75%,31 days 00:00:00
max,33 days 00:00:00


In [148]:
crsp_data = crsp_data.drop(columns=['rf_x'])
crsp_data = crsp_data.rename(columns={"rf_y":"rf"})
crsp_data.head()

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,ticker,mktcap,turnover,adjprc,momentum,short_term_reversal,long_term_reversal,52wk_high,price_rel_52wk_high,rf
0,02742010,60409,20092,0,2,7392.0,1990-01-31,0.25000,0.4375,-0.31250,...,A,3980.31250,0.056685,-0.31250,NaN,0.625000,NaN,NaN,NaN,0.0057
1,02742010,60409,20092,0,2,7392.0,1990-02-28,0.25000,0.4375,0.37500,...,A,4776.37500,0.065243,0.37500,NaN,0.166667,NaN,NaN,NaN,0.0057
2,02742010,60409,20092,0,2,7392.0,1990-03-30,-0.28125,-0.3750,-0.37500,...,A,4776.37500,0.049541,-0.37500,NaN,-0.200000,NaN,NaN,NaN,0.0064
3,02742010,60409,20092,0,2,7392.0,1990-04-30,0.25000,0.3750,-0.28125,...,A,3582.28125,0.055979,-0.28125,NaN,-0.000000,NaN,NaN,NaN,0.0069
4,02742010,60409,20092,0,2,7392.0,1990-05-31,0.18750,-0.3125,0.25000,...,A,3184.25000,0.085577,0.25000,NaN,0.250000,NaN,NaN,NaN,0.0068


In [150]:
del(concat_list)

In [151]:
def get_nan_count(return_series):
        return return_series.isna().sum()


In [152]:
temp = crsp_data.groupby('ticker')['ret'].apply(get_nan_count)
temp.sort_values(ascending=False)

ticker
RFIL    80
HIRE    77
SVA     72
STRP    48
GLMA    39
        ..
DDS      0
PHIO     0
PHLT     0
PHM      0
PTMN     0
Name: ret, Length: 10085, dtype: int64

In [270]:
def compute_momentum(ret):
    """Computes past 11-month cumulative return, skipping the most recent month."""
    
    return  np.exp(np.log(1+ret).rolling(11,min_periods=11).sum().shift(1))-1

crsp_data['momentum'] = crsp_data.groupby('permno')['ret'].transform(compute_momentum)


In [271]:
crsp_data['short_term_reversal'] = -crsp_data['ret'].shift(1)

def compute_long_term_reversal(ret):
    """Computes past 5-year cumulative return, skipping the most recent year."""
    
    return  np.exp(np.log(1+ret).rolling(60,min_periods=60).sum().shift(12))-1

# Compute Long-Term Reversal (-5-Year Return, lagged 1 year)
crsp_data['long_term_reversal'] = crsp_data.groupby('permno')['ret'].transform(compute_long_term_reversal)


def compute_52_week_high(adjprc):
    """Computes past 5-year cumulative return, skipping the most recent year."""
    return adjprc.rolling(60,min_periods=60).max().shift(12)

# Compute Price Relative to 52-Week High
crsp_data['52wk_high'] = crsp_data.groupby('permno')['adjprc'].transform(compute_52_week_high)
crsp_data['price_rel_52wk_high'] = crsp_data['adjprc'] / crsp_data['52wk_high']

In [155]:
crsp_data.head()

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,ticker,mktcap,turnover,adjprc,momentum,short_term_reversal,long_term_reversal,52wk_high,price_rel_52wk_high,rf
0,02742010,60409,20092,0,2,7392.0,1990-01-31,0.25000,0.4375,-0.31250,...,A,3980.31250,0.056685,-0.31250,NaN,NaN,NaN,NaN,NaN,0.0057
1,02742010,60409,20092,0,2,7392.0,1990-02-28,0.25000,0.4375,0.37500,...,A,4776.37500,0.065243,0.37500,NaN,0.166667,NaN,NaN,NaN,0.0057
2,02742010,60409,20092,0,2,7392.0,1990-03-30,-0.28125,-0.3750,-0.37500,...,A,4776.37500,0.049541,-0.37500,NaN,-0.200000,NaN,NaN,NaN,0.0064
3,02742010,60409,20092,0,2,7392.0,1990-04-30,0.25000,0.3750,-0.28125,...,A,3582.28125,0.055979,-0.28125,NaN,-0.000000,NaN,NaN,NaN,0.0069
4,02742010,60409,20092,0,2,7392.0,1990-05-31,0.18750,-0.3125,0.25000,...,A,3184.25000,0.085577,0.25000,NaN,0.250000,NaN,NaN,NaN,0.0068


In [158]:
crsp_data.ticker.nunique()

10085

In [269]:
crsp_data['prc'] = abs(crsp_data['prc'] )

In [159]:
# def compute_rf(group):
#     # Merge with t_bill_data
#     group['date'] = pd.to_datetime(group['date'])
#     print(group.ticker.iloc[0])
#     if group.ticker.iloc[0] == 'SCTTA':
#         print(group.date)
#     # print(group.date)
#     group = group[['date']]
#     merged = pd.merge_asof(
#         group,
#         t_bill_data,
#         on='date',
#         direction='backward'
#     )
#     return merged['rf']
# rf_rates_1m  = crsp_data.groupby('permno').apply(compute_rf).reset_index(level=0, drop=True)
# crsp_data = crsp_data.merge(rf_rates_1m, left_index=True, right_index=True, how='left')

In [254]:
def compute_rolling_metrics(group, window=12):
    """
    Computes rolling beta, idiosyncratic volatility, and unexplained volume for a stock.
    
    Parameters:
        group (DataFrame): Data with 'ret', 'vwretd', 'rf', and 'vol' columns
        window (int): Rolling window size in months
    
    Returns:
        DataFrame: Rolling metrics with NaN for initial window-1 periods
    """
    # Convert to numeric if not already
    ret = pd.to_numeric(group['ret'], errors='coerce')
    rf = pd.to_numeric(group['rf'], errors='coerce')
    vwretd = pd.to_numeric(group['vwretd'], errors='coerce')
    volume = pd.to_numeric(group['vol'], errors='coerce')
    
    # If insufficient data, return DataFrame of NaNs
    if len(group) < window:
        return pd.DataFrame({
            'beta': np.nan,
            'idvol': np.nan,
            'unexp_vol': np.nan
        }, index=group.index)
    
    metrics_list = []
    
    # Calculate rolling metrics
    for i in range(window, len(group) + 1):
        try:
            # Prepare return data
            temp_ret = np.asarray(ret.iloc[i - window:i] - rf.iloc[i-window:i])
            temp_vwretd = np.asarray(vwretd.iloc[i - window:i] - rf.iloc[i-window:i])
            
            # Create design matrix and handle missing values
            X = sm.add_constant(temp_vwretd)
            y = temp_ret
            
            metrics = {}
            
            # Only compute if we have sufficient non-NaN values
            if len(X) >= window-2 and len(y) >= window-2:  # Allow for some missing data
                # Fit market model
                model = sm.OLS(y, X, missing='drop').fit()
                
                # Calculate beta
                metrics['beta'] = model.params[1]
                
                # Calculate idiosyncratic volatility (standard deviation of residuals)
                metrics['idvol'] = np.std(model.resid)
                
                # Calculate unexplained volume if volume data is available
                if volume is not None:
                    temp_vol = volume.iloc[i - window:i]
                    if len(temp_vol) >= window-2:
                        # Detrend volume using log and first difference
                        log_vol = np.log(temp_vol)
                        vol_change = log_vol.diff()
                        
                        # Regress volume changes on absolute market returns
                        X_vol = sm.add_constant(np.abs(temp_vwretd))
                        # print(X_vol.dtype,type(vol_change))
                        vol_model = sm.OLS(np.asarray(vol_change[1:]), X_vol[1:], missing='drop').fit()
                        
                        # Standard deviation of volume regression residuals
                        metrics['unexp_vol'] = np.std(vol_model.resid)
                    else:
                        metrics['unexp_vol'] = np.nan
                else:
                    metrics['unexp_vol'] = np.nan
            else:
                metrics = {
                    'beta': np.nan,
                    'idvol': np.nan,
                    'unexp_vol': np.nan
                }
                
        except (ValueError, np.linalg.LinAlgError):
            metrics = {
                'beta': np.nan,
                'idvol': np.nan,
                'unexp_vol': np.nan
            }
            
        metrics_list.append(metrics)
    
    # Create full DataFrame with NaN padding at start
    nan_rows = [{
        'beta': np.nan,
        'idvol': np.nan,
        'unexp_vol': np.nan
    } for _ in range(window - 1)]
    
    full_df = pd.DataFrame(
        nan_rows + metrics_list,
        index=group.index
    )
    
    return full_df


result = crsp_data.groupby('permno').apply(compute_rolling_metrics).reset_index(level=0, drop=True)

crsp_data = crsp_data.merge(result, left_index=True, right_index=True, how='left')

/Users/jandh/Desktop/Old Desktop/od/2 Quarter/Flexible/Factor-Research/.venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jandh/Desktop/Old Desktop/od/2 Quarter/Flexible/Factor-Research/.venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jandh/Desktop/Old Desktop/od/2 Quarter/Flexible/Factor-Research/.venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jandh/Desktop/Old Desktop/od/2 Quarter/Flexible/Factor-Research/.venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jandh/Desktop/Old Desktop/od/2 Quarte

In [256]:
crsp_data[crsp_data.ticker=="AAPL"]

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,long_term_reversal,52wk_high,price_rel_52wk_high,rf,beta_x,idvol_x,unexp_vol_x,beta_y,idvol_y,unexp_vol_y
3787,03783310,14593,7,8,3,3571.0,1990-01-31,32.37500,38.00000,34.000,...,NaN,NaN,NaN,0.0057,NaN,NaN,NaN,NaN,NaN,NaN
3788,03783310,14593,7,8,3,3571.0,1990-02-28,33.00000,35.00000,34.000,...,NaN,NaN,NaN,0.0057,NaN,NaN,NaN,NaN,NaN,NaN
3789,03783310,14593,7,8,3,3571.0,1990-03-30,33.75000,42.37500,40.250,...,NaN,NaN,NaN,0.0064,NaN,NaN,NaN,NaN,NaN,NaN
3790,03783310,14593,7,8,3,3571.0,1990-04-30,38.75000,43.75000,39.375,...,NaN,NaN,NaN,0.0069,NaN,NaN,NaN,NaN,NaN,NaN
3791,03783310,14593,7,8,3,3571.0,1990-05-31,39.50000,42.62500,41.250,...,NaN,NaN,NaN,0.0068,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4202,03783310,14593,7,8,3,3571.0,2024-08-30,207.23000,229.78999,229.000,...,2.450649,1700.16004,0.134693,0.0048,NaN,NaN,NaN,0.976685,0.052552,0.170252
4203,03783310,14593,7,8,3,3571.0,2024-09-30,216.32001,233.00000,233.000,...,2.170981,1700.16004,0.137046,0.004,NaN,NaN,NaN,0.763847,0.050452,0.167586
4204,03783310,14593,7,8,3,3571.0,2024-10-31,221.69000,236.48000,225.910,...,2.262259,1700.16004,0.132876,0.0039,NaN,NaN,NaN,0.901130,0.050927,0.188185
4205,03783310,14593,7,8,3,3571.0,2024-11-29,222.00999,237.33000,237.330,...,3.434618,1700.16004,0.139593,0.004,NaN,NaN,NaN,0.663048,0.049729,0.188319


In [257]:
crsp_data = crsp_data.drop(columns=["beta_x",	"idvol_x",	"unexp_vol_x"])
crsp_data = crsp_data.rename(columns={"beta_y":	"beta","idvol_y": "idvol",	"unexp_vol_y":"unexp_vol"})
crsp_data[crsp_data.ticker=="AAPL"]

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,adjprc,momentum,short_term_reversal,long_term_reversal,52wk_high,price_rel_52wk_high,rf,beta,idvol,unexp_vol
3787,03783310,14593,7,8,3,3571.0,1990-01-31,32.37500,38.00000,34.000,...,3808.00,NaN,-0.143652,NaN,NaN,NaN,0.0057,NaN,NaN,NaN
3788,03783310,14593,7,8,3,3571.0,1990-02-28,33.00000,35.00000,34.000,...,3808.00,NaN,0.035461,NaN,NaN,NaN,0.0057,NaN,NaN,NaN
3789,03783310,14593,7,8,3,3571.0,1990-03-30,33.75000,42.37500,40.250,...,4508.00,NaN,-0.003235,NaN,NaN,NaN,0.0064,NaN,NaN,NaN
3790,03783310,14593,7,8,3,3571.0,1990-04-30,38.75000,43.75000,39.375,...,4410.00,NaN,-0.183824,NaN,NaN,NaN,0.0069,NaN,NaN,NaN
3791,03783310,14593,7,8,3,3571.0,1990-05-31,39.50000,42.62500,41.250,...,4620.00,NaN,0.021739,NaN,NaN,NaN,0.0068,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4202,03783310,14593,7,8,3,3571.0,2024-08-30,207.23000,229.78999,229.000,...,229.00,0.186703,-0.054411,2.450649,1700.16004,0.134693,0.0048,0.976685,0.052552,0.170252
4203,03783310,14593,7,8,3,3571.0,2024-09-30,216.32001,233.00000,233.000,...,233.00,0.344220,-0.032286,2.170981,1700.16004,0.137046,0.004,0.763847,0.050452,0.167586
4204,03783310,14593,7,8,3,3571.0,2024-10-31,221.69000,236.48000,225.910,...,225.91,0.371223,-0.017467,2.262259,1700.16004,0.132876,0.0039,0.901130,0.050927,0.188185
4205,03783310,14593,7,8,3,3571.0,2024-11-29,222.00999,237.33000,237.330,...,237.33,0.193745,0.030429,3.434618,1700.16004,0.139593,0.004,0.663048,0.049729,0.188319


In [278]:
crsp_data.to_csv("../data/wrds_processed_features.csv")